In [7]:
import imaplib
import pyzmail
import requests
import email
import outlook
import re
import openai

# Set up OpenAI API credentials
openai.api_key = 

In [18]:
mailbox = outlook.Outlook()
mailbox.login('budget2396@outlook.com','')
mailbox.inbox()

unread_ids = mailbox.unreadIds()
matching_ids = mailbox.getIdswithWord(unread_ids, 'transaction')  # Pass the correct iterable to the method

# Regular expression patterns
amount_pattern = r'\$(\d+\.\d{2})'
#vendor_pattern = r'\*\s([\w\s]+)'
vendor_pattern = r"with\s(.+)"


# Extract transaction amount and vendor name
transaction_info = []

for id in unread_ids:
    mailbox.getEmail(id)
    subject_line = mailbox.mailsubject()
    amount_match = re.search(amount_pattern, subject_line)
    vendor_match = re.search(vendor_pattern, subject_line)
    if amount_match and vendor_match:
        amount = amount_match.group(1)
        vendor = vendor_match.group(1)
        print(amount)
        print(vendor)
        transaction_info.append((amount, vendor))

print(transaction_info)



Hello World
 > Signed in as budget2396@outlook.com [b'LOGIN completed.']
15.97
SQ *ZEDS ICE CREAM
36.82
SQ *TFB OPERATIONS L
40.00
HCTRA EZ TAG Rebill
21.65
SQ *BOULDIN ACRES
9.80
P. TERRY'S STAND #13
[('15.97', 'SQ *ZEDS ICE CREAM'), ('36.82', 'SQ *TFB OPERATIONS L'), ('40.00', 'HCTRA EZ TAG Rebill'), ('21.65', 'SQ *BOULDIN ACRES'), ('9.80', "P. TERRY'S STAND #13")]


In [37]:
# Use GPT-3.5 to determine the type of vendor
for amount, vendor in transaction_info:
    gpt_prompt = f"What type of vendor is '{vendor}'? Classify the vendor into a broad financing budget category used for personal finance applications. Only return the specific category."
    response = openai.Completion.create(
        engine='text-davinci-003',
        prompt=gpt_prompt,
        max_tokens=100,
        n=1,
        stop=None,
        temperature=0.3
    )
    vendor_type = response.choices[0].text.strip()
    
    print(f"Transaction Amount: ${amount}")
    print(f"Vendor Name: {vendor}")
    print(f"Vendor Type: {vendor_type}")
    print()

Transaction Amount: $15.97
Vendor Name: SQ *ZEDS ICE CREAM
Vendor Type: Food Vendor

Transaction Amount: $36.82
Vendor Name: SQ *TFB OPERATIONS L
Vendor Type: Financial Institution

Transaction Amount: $40.00
Vendor Name: HCTRA EZ TAG Rebill
Vendor Type: Transportation

Transaction Amount: $21.65
Vendor Name: SQ *BOULDIN ACRES
Vendor Type: Grocery Store

Transaction Amount: $9.80
Vendor Name: P. TERRY'S STAND #13
Vendor Type: Fast Food Restaurant

